In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import TargetEncoder
import numpy as np


In [2]:
# abrir archivo train.jsonlines
leer = pd.read_json('train.jsonlines', lines=True)

df = pd.json_normalize(leer.to_dict(orient='records'))

df

,warranty,sub_status,condition,deal_ids,base_price,non_mercado_pago_payment_methods,seller_id,variations,site_id,listing_type_id,...,seller_address.state.id,seller_address.city.name,seller_address.city.id,shipping.local_pick_up,shipping.methods,shipping.tags,shipping.free_shipping,shipping.mode,shipping.dimensions,shipping.free_methods
0,Calidad y mejor precio,[],new,[],15.02,"[{'description': 'Transferencia bancaria', 'id...",4086929269,[],MLA,bronze,...,AR-B,Quilmes,TUxBQ1FVSWI1MzY,True,[],[],False,not_specified,None,NaN
1,6 meses,[],new,[],5520.00,[],7222239700,[],MLA,bronze,...,AR-X,Córdoba,TUxBQ0NBUGNiZGQx,True,[],[],False,not_specified,None,NaN
2,None,[],used,[],100.00,"[{'description': 'Transferencia bancaria', 'id...",1012912452,[{'attribute_combinations': [{'value_id': '920...,MLA,free,...,AR-B,Jose Marmol,,True,[],[],False,not_specified,None,NaN
3,Dada por la trayectoria de la marca,[],used,[],65.00,"[{'description': 'Transferencia bancaria', 'id...",4449250674,[{'attribute_combinations': [{'value_id': '920...,MLA,free,...,AR-C,Villa Crespo,TUxBQlZJTDQyMjBa,False,[],[],False,custom,None,NaN
4,None,[],new,[],45.00,[],5964151586,[],MLA,bronze,...,AR-C,caba,,True,[],[],False,not_specified,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,Sin garantía,[],used,[],35.00,"[{'description': 'Transferencia bancaria', 'id...",1365962920,[],MLA,bronze,...,AR-B,lanus,,True,[],[],False,not_specified,None,NaN
69996,LIBROS NUEVOS ORIGINALES,[],new,[],249.00,[],9197076837,[],MLA,silver,...,AR-C,CABA,,True,[],[],False,me2,None,NaN
69997,Sí,[],used,[],75.00,[],5839441006,[],MLA,bronze,...,AR-C,Capital Federal,,True,[],[],False,me2,None,NaN
69998,Garantía de fábrica,[],new,[],1980.00,"[{'description': 'Transferencia bancaria', 'id...",9608604363,[],MLA,gold_special,...,AR-C,Liniers,TUxBQkxJTjEzNTha,True,[],[],False,custom,None,NaN


In [3]:
attribute_flat = pd.json_normalize(df['attributes'].explode().dropna().to_list())
df = df.drop(columns=['attributes']).join(attribute_flat,rsuffix='_attribute') 

non_mercado_pago_payment_methods_flat = pd.json_normalize(df['non_mercado_pago_payment_methods'].explode().dropna().to_list())
df = df.drop(columns=['non_mercado_pago_payment_methods']).join(non_mercado_pago_payment_methods_flat,rsuffix='non_mercado_pago_payment_methods')

descriptions_flat = pd.json_normalize(df['descriptions'].explode().dropna().to_list())
df = df.drop(columns=['descriptions']).join(descriptions_flat,rsuffix='descriptions')
df = df.drop(columns=['sub_status','deal_ids','variations','site_id','tags','listing_source','coverage_areas','last_updated','international_delivery_mode','pictures','id','differential_pricing','original_price','currency_id','thumbnail','secure_thumbnail','video_id','catalog_product_id','subtitle','permalink','seller_address.country.name','seller_address.country.id','seller_address.state.id','seller_address.city.id','shipping.methods','shipping.tags','shipping.dimensions','shipping.free_methods','value_id','id_attribute','idnon_mercado_pago_payment_methods','stop_time','start_time'])
df['seller_id'] = df['seller_id'].astype('category')
df['warranty'] = df['warranty'].replace(['si', 'sí','SI','Sí','Si.','SI Y MI REPUTACION','100 %','Yes','Sí.','SÍ'], 'si')
df['warranty'] = df['warranty'].str.replace(" ", "")

df

,warranty,condition,base_price,seller_id,listing_type_id,price,buying_mode,parent_item_id,category_id,official_store_id,...,seller_address.city.name,shipping.local_pick_up,shipping.free_shipping,shipping.mode,attribute_group_id,name,value_name,attribute_group_name,description,type
0,Calidadymejorprecio,new,15.02,4086929269,bronze,15.02,buy_it_now,MLA8336077177,MLA73721,NaN,...,Quilmes,True,False,not_specified,DFLT,Género,Mujer,Otros,Transferencia bancaria,G
1,6meses,new,5520.00,7222239700,bronze,5520.00,buy_it_now,None,MLA373652,NaN,...,Córdoba,True,False,not_specified,FIND,Season,All-Season,Ficha técnica,Acordar con el comprador,G
2,None,used,100.00,1012912452,free,100.00,buy_it_now,MLA2970364564,MLA119365,NaN,...,Jose Marmol,True,False,not_specified,DFLT,Género,Niños,Otros,Efectivo,G
3,Dadaporlatrayectoriadelamarca,used,65.00,4449250674,free,65.00,buy_it_now,MLA8680365117,MLA119482,NaN,...,Villa Crespo,False,False,custom,FIND,Season,All-Season,Ficha técnica,Transferencia bancaria,G
4,None,new,45.00,5964151586,bronze,45.00,buy_it_now,MLA8281770093,MLA3530,NaN,...,caba,True,False,not_specified,DFLT,Género,Mujer,Otros,Efectivo,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,Singarantía,used,35.00,1365962920,bronze,35.00,auction,MLA6505565736,MLA41287,NaN,...,lanus,True,False,not_specified,COMODIDADES,Calefacción,Sí,Comodidades,Efectivo,G
69996,LIBROSNUEVOSORIGINALES,new,249.00,9197076837,silver,249.00,buy_it_now,MLA1381768684,MLA41149,NaN,...,CABA,True,False,me2,COMODIDADES,Cancha de fútbol,No,Comodidades,Transferencia bancaria,G
69997,si,used,75.00,5839441006,bronze,75.00,buy_it_now,MLA7048318102,MLA41184,NaN,...,Capital Federal,True,False,me2,COMODIDADES,Cancha de tenis,No,Comodidades,Tarjeta de crédito,N
69998,Garantíadefábrica,new,1980.00,9608604363,gold_special,1980.00,buy_it_now,MLA5217059313,MLA351716,NaN,...,Liniers,True,False,custom,COMODIDADES,Conexión a internet,No,Comodidades,Acordar con el comprador,G


In [4]:
#df.to_csv('train.csv', index=False)

In [6]:
df['official_store_id'].value_counts()

official_store_id
1580.0    25
6448.0    20
9218.0    16
4701.0    16
5192.0    16
          ..
5882.0     1
7715.0     1
9309.0     1
6335.0     1
6326.0     1
Name: count, Length: 164, dtype: int64

In [5]:
# te = TargetEncoder()
# le = LabelEncoder()

# cat = df.select_dtypes(include=['object','string']).drop(columns=['condition'])
# cat
# df[cat.columns] = te.fit_transform(df[cat.columns], df['condition'])

# df['condition'] = le.fit_transform(df['condition'])

In [6]:
X = df.drop(['condition'], axis=1)
y = df['condition']

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
rf_model = RandomForestClassifier(n_estimators=431, random_state=16,max_depth=86) #434 mejor resultado

encoder = Pipeline(steps=[('target encoder', TargetEncoder(smooth='auto'))])

pipeline = Pipeline(steps=[('preprocessor', encoder), ('classifier', rf_model)])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)


In [8]:
lecture = pd.read_json('test.jsonlines', lines=True)
test = pd.json_normalize(lecture.to_dict(orient='records'))

In [9]:
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy:.4f}")
# confusion matrix

print(classification_report(y_val, y_pred))


Accuracy: 0.9131
              precision    recall  f1-score   support

         new       0.94      0.90      0.92      7432
        used       0.89      0.93      0.91      6568

    accuracy                           0.91     14000
   macro avg       0.91      0.91      0.91     14000
weighted avg       0.91      0.91      0.91     14000



In [10]:
attribute_flat = pd.json_normalize(test['attributes'].explode().dropna().to_list())
test = test.drop(columns=['attributes']).join(attribute_flat,rsuffix='_attribute') 

non_mercado_pago_payment_methods_flat = pd.json_normalize(test['non_mercado_pago_payment_methods'].explode().dropna().to_list())
test = test.drop(columns=['non_mercado_pago_payment_methods']).join(non_mercado_pago_payment_methods_flat,rsuffix='non_mercado_pago_payment_methods')

descriptions_flat = pd.json_normalize(test['descriptions'].explode().dropna().to_list())
test = test.drop(columns=['descriptions']).join(descriptions_flat,rsuffix='descriptions')
test = test.drop(columns=['sub_status','deal_ids','variations','site_id','tags','listing_source','coverage_areas','last_updated','international_delivery_mode','pictures','id','differential_pricing','original_price','currency_id','thumbnail','secure_thumbnail','video_id','catalog_product_id','subtitle','permalink','seller_address.country.name','seller_address.country.id','seller_address.state.id','seller_address.city.id','shipping.methods','shipping.tags','shipping.dimensions','shipping.free_methods','value_id','id_attribute','idnon_mercado_pago_payment_methods','stop_time','start_time'])
test['warranty'] = test['warranty'].replace(['si', 'sí','SI','Sí','Si.','SI Y MI REPUTACION','100 %','Yes','Sí.','SÍ'], 'si')
test['warranty'] = test['warranty'].str.replace(" ", "")
test['seller_id'] = test['seller_id'].astype('category')

In [11]:
y_pred_test = pipeline.predict(test)

In [12]:
id_col = range(1, len(y_pred_test) + 1)

# Crear el DataFrame
df = pd.DataFrame({'ID': id_col, 'condition': y_pred_test})

df.to_csv('ElPajarito.csv', index=False)